This notebook generates videos from cilia simulation data.

**Features:**
- Generate top-down videos colored by phase
- Specify duration and time range
- Automatic frame rate calculation
- Progress bar during rendering

**Output:**
- MP4 video saved to `analysis_output/` directory
- High-quality rendering (180 DPI)

In [ ]:
# Import libraries
import sys
sys.path.append('.')
from plotting_and_analysis_functions import *
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# Define the simulation path
# Adjust the date and path according to your actual simulation
sim_path = '../data/spacing_e4/20251027/ciliate_216fil_40962blob_8.00R_0.1500torsion_0.2182tilt_0.3000f_eff_1.4960theta0_0.0000freqshift'

# Parameters (adjust these to match your simulation)
filament_length = 49.4      # Filament length in simulation units (L)
num_steps = 500             # Steps per period
sphere_radius = 197.6       # Sphere radius in simulation units
num_segs = 20               # Number of segments per filament

In [ ]:
# Load the simulation data
print("=" * 60)
print("LOADING SIMULATION DATA")
print("=" * 60)
sim = load_simulation(sim_path, num_steps=num_steps, sphere_radius=sphere_radius, num_segs=num_segs)
print(f"\n✅ Successfully loaded simulation:")
print(f"   • Timesteps: {sim.phases.shape[0]}")
print(f"   • Filaments: {sim.phases.shape[1]}")
print(f"   • Segments per filament: {sim.num_segs}")
print(f"   • Sphere radius: {sim.sphere_radius:.2f} units = {sim.sphere_radius/filament_length:.2f} L")
print(f"   • Filament length: {filament_length:.2f} units (L)")
print(f"   • Time duration: {sim.times[-1]:.2f} periods")
print(f"   • Steps per period: {num_steps}")
print("=" * 60)

In [ ]:
# Video configuration
video_duration = 15.0       # Duration of video in seconds
n_periods = 15              # Number of periods to include in video
target_fps = 25             # Target frames per second for smooth playback

# Calculate which frames to use
total_frames = sim.phases.shape[0]
total_periods = sim.times[-1]

# Select last n_periods
if n_periods > total_periods:
    print(f"[warn] Requested {n_periods} periods, but simulation only has {total_periods:.2f} periods.")
    print(f"[warn] Using entire simulation instead.")
    start_period = 0
    end_period = total_periods
else:
    start_period = total_periods - n_periods
    end_period = total_periods

# Find frame indices corresponding to desired period range
start_frame = np.searchsorted(sim.times, start_period)
end_frame = np.searchsorted(sim.times, end_period)

# Calculate stride to achieve target video duration
frames_in_range = end_frame - start_frame
stride = max(1, int(frames_in_range / (video_duration * target_fps)))
actual_fps = frames_in_range / (stride * video_duration)

print("\n" + "=" * 60)
print("VIDEO CONFIGURATION")
print("=" * 60)
print(f"Video duration:     {video_duration:.1f} seconds")
print(f"Period range:       {start_period:.2f} to {end_period:.2f} (last {n_periods} periods)")
print(f"Frame range:        {start_frame} to {end_frame} ({frames_in_range} frames)")
print(f"Stride:             {stride} (using every {stride} frame(s))")
print(f"Target FPS:         {target_fps}")
print(f"Actual FPS:         {actual_fps:.1f}")
print(f"Video frames:       {frames_in_range // stride}")
print("=" * 60)

In [ ]:
# Create subset of simulation data for the video
sim_subset = SimulationData(
    base_path=sim.base_path,
    phases=sim.phases[start_frame:end_frame],
    phases_raw=sim.phases_raw[start_frame:end_frame],
    times=sim.times[start_frame:end_frame],
    seg_positions=sim.seg_positions[start_frame:end_frame],
    basal_pos=sim.basal_pos,
    basal_phi=sim.basal_phi,
    order_idx=sim.order_idx,
    num_segs=sim.num_segs,
    num_steps=sim.num_steps,
    num_fils=sim.num_fils,
    sphere_radius=sim.sphere_radius
)

print(f"\n✅ Created subset simulation:")
print(f"   • Time range: {sim_subset.times[0]:.2f} to {sim_subset.times[-1]:.2f} periods")
print(f"   • Number of frames: {sim_subset.phases.shape[0]}")
print(f"   • Duration: {n_periods} periods")


In [ ]:
# Output path
out_dir = Path("analysis_output/videos")
out_dir.mkdir(exist_ok=True, parents=True)
out_path = out_dir / f"{Path(sim_path).name}_last{n_periods}periods_{video_duration}s.mp4"

print("\n" + "=" * 60)
print("GENERATING VIDEO")
print("=" * 60)
print(f"Output: {out_path}")
print(f"Rendering {frames_in_range // stride} frames at {actual_fps:.1f} FPS...")
print("This may take several minutes...")
print("=" * 60 + "\n")

# Generate video
try:
    video_path = make_topdown_video(
        base_path=sim_path,
        sim=sim_subset,
        num_steps=num_steps,
        out_path=str(out_path),
        stride=stride,
        fps=int(actual_fps),
        cmap=DEFAULT_CMAP,
        progress=True,
        dpi=180
    )
    
    print("\n" + "=" * 60)
    print("✅ VIDEO GENERATION COMPLETE!")
    print("=" * 60)
    print(f"Video saved to: {video_path}")
    print(f"Duration: {video_duration} seconds")
    print(f"Frame rate: {actual_fps:.1f} FPS")
    print(f"Resolution: 1080p (180 DPI)")
    print("=" * 60)
    
except Exception as e:
    print("\n" + "=" * 60)
    print("❌ VIDEO GENERATION FAILED")
    print("=" * 60)
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()
    print("=" * 60)


## Summary

**Video Parameters:**
- **Duration**: 15 seconds
- **Time range**: Last 15 periods of the simulation
- **Frame rate**: ~25 FPS (adjusted based on available frames)
- **Resolution**: High quality (180 DPI)
- **View**: Top-down (viewing from above the sphere)
- **Coloring**: Phase-based (using cyclic colormap)

**Output Location:**
- Videos are saved to `analysis_output/videos/` directory
- Filename includes simulation parameters and time range

**Performance Notes:**
- Rendering time depends on number of frames and system performance
- Progress bar shows rendering status
- Typical rendering time: 5-15 minutes for 15 seconds of video

**Adjustable Parameters:**
- `video_duration`: Length of output video in seconds
- `n_periods`: Number of beat cycles to include
- `target_fps`: Desired frames per second (higher = smoother but longer to render)
- `dpi`: Resolution quality (higher = better quality but larger file size)
- `stride`: Frame sampling rate (automatically calculated, but can be manually adjusted)

**Tips:**
- Increase `target_fps` for smoother motion (but longer rendering time)
- Decrease `stride` to include more frames (but increases video file size)
- Adjust `dpi` to balance quality vs. file size
- Use `n_periods` to focus on specific portions of the simulation